# RAG

Implement a base RAG module in DSPy. 
Given a question, retrieve the top-k documents in a list of HTML documents, then pass them as context to an LLM.

Refer to https://dspy.ai/tutorials/rag/. 


In [1]:
import dspy
from sentence_transformers import SentenceTransformer

# Load an extremely efficient local model for retrieval
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")

# Create an embedder using the model's encode method
embedder = dspy.Embedder(model.encode)

# Traverse a directory and read html files - extract text from the html files
import os
from bs4 import BeautifulSoup
def read_html_files(directory):
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                texts.append(soup.get_text())
    return texts

In [2]:
corpus = read_html_files("../PragmatiCQA-sources/The Legend of Zelda")
print(f"Loaded {len(corpus)} documents. Will encode them below.")

Loaded 406 documents. Will encode them below.


In [11]:
# Parameters for the retriever
max_characters = 10000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

search("What is the name of Link's horse?").passages


['\n\n\n\n\n\n\n\n      Link\'s Uncle\n     \n\n\n\n\n\n\n         Artwork\n        \n\n         Sprite\n        \n\n\n\n\n\n\n            ALttP\n           \n\n\n\n\n\n\n\n            ALttP\n           \n\n\n\n\n\n\n\n\n\n      Title(s)\n     \n\n\n       Royal Knight\n      \n\n\n\n\n      Race\n     \n\n\n       Hylian\n      \n\n\n\n\n      Main appearance(s)\n     \n\n\n\n\n         A Link to the Past\n        \n\n\n\n\n\n\n      Other appearance(s)\n     \n\n\n\n        A Link to the Past\n       \n       (Ishinomori)\n      \n\n\n\n\n      Era(s)\n     \n\n\n       Era of Light and Dark\n      \n\n\n\n\n      Family\n     \n\n\n\n        Link\n       \n\n        (nephew)\n       \n\n\n\n\n\n\n\n    Link\'s Uncle\n   \n   is a character in\n   \n\n     A Link to the Past\n    \n\n   .\n  \n\n\n\n     Contents\n    \n\n\n\n\n\n       1\n      \n\n       Biography\n      \n\n\n\n\n\n       2\n      \n\n       Other Appearances\n      \n\n\n\n\n\n         2.1\n        \n\n\n        

In [4]:
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
lm = dspy.LM('xai/grok-3-mini')
dspy.configure(lm=lm)

In [ ]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)
    
rag = RAG()

In [10]:
answer = rag(question="What is the main plot of The Legend of Zelda?")  # Example query

print(answer.response)  # Print the response from the RAG model

The main plot of The Legend of Zelda follows Link, a young hero, as he embarks on a quest to rescue Princess Zelda and defeat the evil Ganon. Ganon has stolen the Triforce of Power and seeks the Triforce of Wisdom to conquer Hyrule. Zelda breaks the Triforce of Wisdom into eight fragments and hides them to prevent Ganon's success, then sends her nursemaid Impa to find a hero. Link collects the fragments, explores dungeons, battles enemies, and ultimately confronts Ganon in his lair on Death Mountain to restore peace to the kingdom.


In [9]:
q = 'Can you name the games in the Legend of Zelda series?' 

print(rag(question=q).response)

2025/08/23 12:45:37 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/23 12:45:37 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


The main games in the Legend of Zelda series, based on the provided context, are:
- The Legend of Zelda (1986)
- The Adventure of Link (1987)
- A Link to the Past (1991)
- Link's Awakening (1993)
- Ocarina of Time (1998)
- Majora's Mask (2000)
- Oracle of Ages (2001)
- Oracle of Seasons (2001)
- Four Swords (2002)
- The Wind Waker (2002)
- Four Swords Adventures (2004)
- The Minish Cap (2004)
- Twilight Princess (2006)
- Phantom Hourglass (2007)
- Spirit Tracks (2009)
- Skyward Sword (2011)
- A Link Between Worlds (2013)
- Tri Force Heroes (2015)
- Breath of the Wild (2017)
